### PCA + ANN + XGB

In [ ]:
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.datasets import load_wine

In [ ]:
data = pd.read_csv('/content/allData800 final.csv')

In [ ]:
df=pd.get_dummies(data,columns=['classification'],drop_first=True)

In [ ]:
df.head()

,age,gender,vol,sg,freq,sod,pot,chlo,phos,prot,...,urea,creatinine,bun,uricacid,rbc,wbc,pcv,pe,ane,classification_notckd
0,45,1,2.0,1.013,0,138,3.4,96,6.2,7.0,...,6.0,1.2,8.0,5.3,5.1,5.2,0.38,0,0,1
1,56,1,2.2,1.011,0,140,3.1,100,3.8,6.9,...,8.9,1.0,14.0,7.4,5.7,6.1,0.40,0,0,1
2,47,1,2.0,1.021,0,145,4.6,98,4.1,8.0,...,5.6,0.9,12.0,4.3,6.1,6.0,0.46,0,0,1
3,42,1,2.3,1.001,0,138,4.0,101,5.9,6.8,...,10.4,0.8,16.0,8.3,5.6,5.8,0.39,0,0,1
4,45,0,2.0,1.015,0,141,4.9,98,3.9,7.2,...,12.8,0.8,11.4,4.9,5.1,6.5,0.41,0,0,1


In [ ]:
X = df.drop('classification_notckd', axis=1)
y = df['classification_notckd']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
exp = pca.explained_variance_ratio_
exp

array([0.3179018 , 0.12989466, 0.07707416, 0.06802061, 0.06035462,
       0.04987878, 0.04823962, 0.0383757 , 0.03641488, 0.03352441,
       0.02721327, 0.02296425, 0.02135163, 0.01911703, 0.01375396,
       0.01107467, 0.00958513, 0.00508235, 0.00433416, 0.00332596,
       0.00251834])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
print(X_train.shape)

(560, 3)


In [ ]:
y_train.dtype

dtype('uint8')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import xgboost as xgb
from sklearn.metrics import accuracy_score
import seaborn as sns

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(units=64, activation='relu', input_dim=3))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(units=32, activation='relu'))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 1s 25ms/step - loss: 0.6132 - accuracy: 0.7321 - val_loss: 0.6015 - val_accuracy: 0.6542
Epoch 2/10
9/9 [==============================] - 0s 9ms/step - loss: 0.5183 - accuracy: 0.7768 - val_loss: 0.5737 - val_accuracy: 0.6708
Epoch 3/10
9/9 [==============================] - 0s 8ms/step - loss: 0.4712 - accuracy: 0.8036 - val_loss: 0.5716 - val_accuracy: 0.6958
Epoch 4/10
9/9 [==============================] - 0s 7ms/step - loss: 0.4623 - accuracy: 0.8036 - val_loss: 0.5773 - val_accuracy: 0.6917
Epoch 5/10
9/9 [==============================] - 0s 8ms/step - loss: 0.4670 - accuracy: 0.8000 - val_loss: 0.5807 - val_accuracy: 0.7083
Epoch 6/10
9/9 [==============================] - 0s 8ms/step - loss: 0.4559 - accuracy: 0.8179 - val_loss: 0.5795 - val_accuracy: 0.7167
Epoch 7/10
9/9 [==============================] - 0s 8ms/step - loss: 0.4494 - accuracy: 0.8179 - val_loss: 0.5739 - val_accuracy: 0.7167
Epoch 8/10
9/9 [=================

In [ ]:
ann_features_train = ann_model.predict(X_train)
ann_features_test = ann_model.predict(X_test)

8/8 [==============================] - 0s 2ms/step


In [ ]:
print(ann_features_train.shape, ann_features_test.shape)

(560, 1) (240, 1)


In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(ann_features_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
xgb_predictions = xgb_model.predict(ann_features_test)

In [ ]:
accuracy = accuracy_score(y_test, xgb_predictions)
print("XGBoost Model Accuracy:", accuracy)

XGBoost Model Accuracy: 0.8625


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [ ]:
precision = precision_score(y_test, xgb_predictions)
recall = recall_score(y_test, xgb_predictions)
f1 = f1_score(y_test, xgb_predictions)
roc_auc = roc_auc_score(y_test, xgb_predictions)
confusion = confusion_matrix(y_test, xgb_predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC-ROC Score:", roc_auc)
print("Confusion Matrix:\n", confusion)

Precision: 0.9
Recall: 0.8689655172413793
F1 Score: 0.8842105263157896
AUC-ROC Score: 0.8607985480943738
Confusion Matrix:
 [[ 81  14]
 [ 19 126]]


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 2, 3]
}


In [ ]:
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

In [ ]:
grid_search.fit(ann_features_train, y_train)

Fitting 5 folds for each of 5184 candidates, totalling 25920 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.1, 0.2, 0.3],
                         'max_depth': [3, 4, 5, 6],
                         'min_child_weight': [1, 2, 3],
                         'n_estimators': [100, 200, 300, 400],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='accuracy', verbose=1)

In [ ]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.3, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 400, 'subsample': 0.8}


In [ ]:
best_xgb_model = grid_search.best_estimator_

In [ ]:
model_filename = "best_xgb_model.pkl"

In [ ]:
import joblib

In [ ]:
joblib.dump(best_xgb_model, model_filename)

['best_xgb_model.pkl']

In [ ]:
# load model later
# loaded_model = joblib.load(model_filename)

In [ ]:
xgb_predictions = best_xgb_model.predict(ann_features_test)

In [ ]:
accuracy = accuracy_score(y_test, xgb_predictions)
print("Tuned XGBoost Model Accuracy:", accuracy)

Tuned XGBoost Model Accuracy: 0.875


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(best_xgb_model, ann_features_test, y_test, cv=5)
print(np.mean(scores))
print(scores)

0.8833333333333332
[0.91666667 0.83333333 0.875      0.91666667 0.875     ]


In [ ]:
precision = precision_score(y_test, xgb_predictions)
recall = recall_score(y_test, xgb_predictions)
f1 = f1_score(y_test, xgb_predictions)
roc_auc = roc_auc_score(y_test, xgb_predictions)
confusion = confusion_matrix(y_test, xgb_predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC-ROC Score:", roc_auc)
print("Confusion Matrix:\n", confusion)

Precision: 0.896551724137931
Recall: 0.896551724137931
F1 Score: 0.896551724137931
AUC-ROC Score: 0.8693284936479129
Confusion Matrix:
 [[ 80  15]
 [ 15 130]]
